In [1]:
import pandas as pd
import numpy as np
import random
import math
import timeit
import itertools
import warnings
import pickle
import gc
import sys
import matplotlib.pyplot as plt
from os.path import join, isfile
import xml.etree.ElementTree as ET
from collections import Counter
warnings.filterwarnings('ignore')
np.set_printoptions(suppress=True, formatter={'float': lambda x: "{0:0.2f}".format(x)})
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))

In [2]:
mainPath = "../data"
beacons = join(mainPath, "beacon")
models = join(mainPath, "models")
ceuPath = join(beacons, "CEU")
opensnpPath = join(beacons, "OpenSNP")
utahPath = join(beacons, "Utah")

# CEU

In [3]:
smallBeacon = pd.read_csv(join(ceuPath, "Beacon_65.txt"), index_col=0, delim_whitespace=True)
extraBeacon = pd.read_csv(join(ceuPath, "Beacon_99.txt"), index_col=0, delim_whitespace=True)
beacon = pd.read_csv(join(ceuPath, "Beacon_164.txt"), index_col=0, delim_whitespace=True)
maf = pd.read_csv(join(ceuPath, "MAF.txt"), index_col=0, delim_whitespace=True)
reference = pickle.load(open(join(ceuPath, "reference.pickle"),"rb"))
#giant = pd.concat([extra,beacon], axis=1)

In [4]:
chromosome_index = 1
ind = (maf["chromosome"] == "chr"+str(chromosome_index)).values
print(np.sum(ind)," SNP's exist in chromosome ", chromosome_index)

314024  SNP's exist in chromosome  1


In [5]:
reference = reference[ind]
beacon = beacon.loc[ind]
smallBeacon = smallBeacon.loc[ind]
extraBeacon = extraBeacon.loc[ind]
maf = maf.loc[ind]

# UTAH

In [19]:
utah = pd.read_excel(join(utahPath, "UTAH_100000.xlsx"), index_col=0)
utah

,12877,12878,12879,12880,12881,12882,12883,12884,12885,12886,12887,12888,12889,12890,12891,12892
SNP Position,,,,,,,,,,,,,,,,
715348,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
723798,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
754503,1,2,2,2,1,1,2,2,1,1,1,2,2,1,2,2
757640,1,2,2,2,1,1,2,2,1,1,1,2,2,1,2,2
757734,1,2,2,2,1,1,2,2,1,1,1,2,2,1,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249172187,2,1,1,2,1,2,1,2,2,1,1,1,1,1,1,1
249181149,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
249182887,2,1,1,2,1,2,1,2,2,1,1,1,1,2,1,2


In [ ]:
utah = pd.read_csv(join(utahPath, "UTAHFamilyData.txt"), index_col=0, header=None, sep='\t')
utah

#### Small Problem in UTAH Data

In [25]:
# Built rs_id query for dbSNP
query1 = ""
query2 = ""
query3 = ""
query4 = ""
for i in range(len(utah.index.values)//4):
    query1 += "1: " + str(utah.index.values[i]) + " OR " 
    query2 += "1: " + str(utah.index.values[i+len(utah.index.values)//4]) + " OR "        
    query3 += "1: " + str(utah.index.values[i+len(utah.index.values)//2]) + " OR "        
    query4 += "1: " + str(utah.index.values[i+3*len(utah.index.values)//4]) + " OR "        
query1 = query1[:-4]
query2 = query2[:-4]
query3 = query3[:-4]
query4 = query4[:-4]

In [26]:
query1

'1: 715348 OR 1: 723798 OR 1: 754503 OR 1: 757640 OR 1: 757734 OR 1: 757936 OR 1: 758144 OR 1: 758626 OR 1: 760912 OR 1: 761147 OR 1: 763769 OR 1: 772755 OR 1: 775789 OR 1: 777122 OR 1: 778302 OR 1: 780027 OR 1: 785050 OR 1: 790696 OR 1: 790758 OR 1: 799463 OR 1: 804540 OR 1: 812267 OR 1: 812284 OR 1: 812751 OR 1: 827267 OR 1: 827277 OR 1: 827323 OR 1: 830807 OR 1: 838153 OR 1: 842825 OR 1: 846338 OR 1: 846489 OR 1: 846600 OR 1: 847250 OR 1: 849998 OR 1: 851499 OR 1: 851757 OR 1: 852133 OR 1: 852964 OR 1: 857728 OR 1: 858691 OR 1: 858801 OR 1: 859404 OR 1: 859690 OR 1: 859701 OR 1: 859913 OR 1: 860416 OR 1: 860461 OR 1: 860521 OR 1: 860792 OR 1: 861008 OR 1: 861630 OR 1: 861808 OR 1: 862093 OR 1: 862124 OR 1: 862383 OR 1: 862389 OR 1: 862866 OR 1: 863124 OR 1: 864726 OR 1: 864755 OR 1: 864757 OR 1: 864938 OR 1: 866319 OR 1: 866920 OR 1: 867584 OR 1: 867993 OR 1: 868329 OR 1: 868404 OR 1: 868791 OR 1: 868891 OR 1: 868981 OR 1: 870903 OR 1: 876499 OR 1: 877831 OR 1: 879676 OR 1: 879687 O

In [20]:
# Read XML files
a={}
entries = []
errors = []
d = np.array(["A", "T", "C", "G"])
for i in range(1,5):
    root = ET.parse(join(utahPath, "snp_result_"+str(i)+".xml")).getroot()
    for x in root.findall('DocumentSummary'):
        rsId = "rs" + x.attrib["uid"]
        pos = x.findall("CHRPOS_PREV_ASSM")[0].text[2:]
        if int(pos) in a:
            continue
        a[int(pos)] = rsId
        try:
            # MAF entry
            snp = x.findall("GLOBAL_MAFS")[0].findall("MAF")[0].findall("FREQ")[0].text
            freq = float(snp.split("=")[1].split("/")[0])
            minor = snp.split("=")[0]
            major = np.random.choice(np.setdiff1d(d, minor), 1, replace=False)[0]
            entry = np.array(["chr1", pos, rsId, minor, freq, major, 1-freq, freq], dtype=object)
            entries.append(entry)
        except:
            freq = 0.001
            minor = np.random.choice(d,1, replace=False)[0]
            major = np.random.choice(np.setdiff1d(d, minor), 1, replace=False)[0]
            entry = np.array(["chr1", pos, rsId, minor, freq, major, 1-freq, freq], dtype=object)
            entries.append(entry)

entries = np.array(entries)
vals = np.array([a[i] for i in sorted(a)])
keys = np.array(sorted(a))
rsIds = vals[np.in1d(keys, utah.index.values)]

In [24]:
rsIds.shape

(3195,)

In [21]:
utah.index = rsIds
entries = entries[np.in1d(entries[:, 2],utah.index.values)]
entries = entries[entries[:,2].argsort()]
utah.columns = ["UTAH1","UTAH2","UTAH3","UTAH4","UTAH5","UTAH6","UTAH7","UTAH8","UTAH9","UTAH10","UTAH11"]
utah.sort_index(inplace=True)

ValueError: Length mismatch: Expected axis has 100499 elements, new values have 3195 elements

In [214]:
# MAF Edit
mafUtah = pd.DataFrame(data=entries, columns=maf.columns)
x = np.setdiff1d(mafUtah["markerId"].values, maf["markerId"].values)
mafUtah_ = mafUtah[mafUtah["markerId"].isin(x)]

In [226]:
maf_ = pd.concat([maf, mafUtah_], axis=0)
maf_.sort_index(inplace=True)

In [216]:
utah = utah.astype(str)
data = utah.values
for i in range(len(utah)):
    data[i][data[i] == "2"] = mafUtah.iloc[i]["referenceAllele"] + mafUtah.iloc[i]["referenceAllele"]
    data[i][data[i] == "0"] = mafUtah.iloc[i]["otherAllele"] + mafUtah.iloc[i]["otherAllele"]
    data[i][data[i] == "1"] = ''.join(sorted(mafUtah.iloc[i]["referenceAllele"] + mafUtah.iloc[i]["otherAllele"]))    
utah = pd.DataFrame(data=data, columns=utah.columns, index=utah.index)

In [232]:
all_beacon = pd.merge(beacon, utah, left_index=True, right_index=True, how="outer")
all_beacon.fillna("NN", inplace=True)
all_beacon

,NA06986,NA06989,NA06993,NA06994,NA07019,NA07022,NA07045,NA07055,NA07056,NA07347,...,UTAH2,UTAH3,UTAH4,UTAH5,UTAH6,UTAH7,UTAH8,UTAH9,UTAH10,UTAH11
rs1000050,TT,TT,TT,TT,NN,TT,TT,TT,CC,CT,...,NN,NN,NN,NN,NN,NN,NN,NN,NN,NN
rs1000072,NN,NN,GG,GG,GG,GG,NN,GG,GG,NN,...,NN,NN,NN,NN,NN,NN,NN,NN,NN,NN
rs1000073,GG,AG,AG,GG,NN,AG,GG,AA,AG,AG,...,NN,NN,NN,NN,NN,NN,NN,NN,NN,NN
rs1000075,NN,NN,CC,CT,CT,TT,NN,CT,CT,NN,...,NN,NN,NN,NN,NN,NN,NN,NN,NN,NN
rs1000085,GG,CG,GG,CG,NN,GG,GG,GG,CG,GG,...,NN,NN,NN,NN,NN,NN,NN,NN,NN,NN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
rs999918,NN,NN,GG,CC,CG,CG,NN,GG,CC,NN,...,NN,NN,NN,NN,NN,NN,NN,NN,NN,NN
rs999923,CC,CT,TT,TT,NN,CC,CT,CT,TT,TT,...,NN,NN,NN,NN,NN,NN,NN,NN,NN,NN
rs999932,NN,NN,AG,GG,AG,AA,NN,AG,AG,NN,...,NN,NN,NN,NN,NN,NN,NN,NN,NN,NN
rs999936,NN,NN,GG,GG,GG,GG,NN,GG,GG,NN,...,NN,NN,NN,NN,NN,NN,NN,NN,NN,NN


In [239]:
# Save Files
all_beacon.to_csv(join(utahPath, "CombinedBeacon.txt"), sep="\t")
utah.to_csv(join(utahPath, "UTAHBeacon.txt"), sep="\t")
maf_.to_csv(join(utahPath, "CombinedMAF.txt"), sep="\t")